# Imports & Initialization

In [ ]:
# Move to the root
import os
cwd = os.getcwd()
if os.path.basename(cwd) != "cv-in-farming":
    os.chdir("../")
print("Current directory:", os.getcwd())

In [ ]:
from src.dataloader import FurrowDataset
from utils.augment_frames import *
from utils.helpers import coord_to_mask, show_image, show_overlay

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def visualize_item(item):
    shape = (480, 640)
    if "depth_arr" in item:
        depth_arr = np.array(item['depth_arr'])
        depth_arr = np.rint(255 * (depth_arr / depth_arr.max()))
        depth_arr = np.clip(depth_arr * 5, a_min=0, a_max=255).astype(np.uint8)
        show_image(np.array(depth_arr), cmap="gray")
        print(f"Depth array shape: {depth_arr.shape}")

    if "edge_pixels" in item:
        edge_pixels = item['edge_pixels']
        edge_mask = coord_to_mask(shape, edge_pixels, thickness=1)
        show_image(edge_mask, cmap="gray")

    if "rgb_img" in item:
        rgb_img = item['rgb_img']
        show_image(rgb_img)

    if "depth_img" in item:
        depth_img = item['depth_img']
        show_image(depth_img)
        
    if "rgb_img" in item and "edge_pixels" in item:
        rgb_img = item['rgb_img']
        edge_pixels = item['edge_pixels']
        edge_mask = coord_to_mask(shape, edge_pixels, thickness=1)
        show_overlay(np.array(rgb_img), edge_mask)

# Load Dataset

In [ ]:
dataset_args = {
  "data_path": "./dataset/train/20201112_134332",
  "load_edge": True,
  "load_time": False,
}
output_path = "./dataset/train/20201112_134332_aug"
dataset = FurrowDataset(dataset_args)
print(dataset)

## Inspect Dataset (Optional)

In [ ]:
from pprint import pprint
frame_ids = dataset.frame_ids
pprint(frame_ids)

# Augment Multiple Samples

In [ ]:
load_darr = True
load_rgb = True
load_drgb = True
load_edge = True
load_time = False

items = list(range(len(dataset)))
for i in items:
    item = dataset.get_frame_files(i, 
                                   load_darr=load_darr,
                                   load_rgb=load_rgb,
                                   load_drgb=load_drgb,
                                   load_edge=load_edge,
                                   load_time=load_time)
    modified = apply_random_augment(item)
    store_item(modified, output_path)

# Augment A Random Sample

In [ ]:
rand_idx = np.random.randint(0, 100)
load_darr = True
load_rgb = True
load_drgb = True
load_edge = True
load_time = False
item = dataset.get_frame_files(rand_idx, 
                               load_darr=load_darr, 
                               load_rgb=load_rgb, 
                               load_drgb=load_drgb, 
                               load_edge=load_edge, 
                               load_time=load_time)
frame_id = item['frame_id']

print(f"Random Index: {rand_idx} <-> Frame ID: {frame_id}")

In [ ]:
modified = apply_random_augment(item)
store_item(modified, output_path)

In [ ]:
visualize_item(item)
visualize_item(modified)